# Create the Initial Media CSV

In [1]:
cd ../

/Users/zak/Documents/Projects/media_compiler/core


In [2]:
!pwd

/Users/zak/Documents/Projects/media_compiler/core


In [3]:
NUM_MEDIA = 15       # Number of Unique Medias to Create
OVERLAY_VOLUME = 200 # Size Of Overlay
REAGENT_VOLUME = 1000
WELL_VOLUME = 1100
WATER_VOLUME = 1600
EXTRA_WELLS = {'Glucose':4,'NaCl':3,'KH2PO4':2,'Na2HPO4':2}#,'NH4Cl':3,'EDTA':2}
OUTFILE  = 'data/indigoidine_media.csv'
DECKFILE = 'data/indigoidine_wells.csv'
MEDIA_COMPONENTS='data/Indigoidine_Stock_Solutions.csv'

In [4]:
import pandas as pd
from IPython.display import display
from pyDOE import lhs
import numpy as np
import math

In [5]:
def get_well_num(well_address,plate='96_well'):
    row = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    row_num = lambda x: row.find(x)
    if plate == '96_well':
        wells_per_row = 12
        return (row_num(well_address[0]))*wells_per_row + int(well_address[1:])
    else:
        raise 'Need to Implement other Plate Sizes'
        
print(get_well_num('D2'))

38


In [6]:
media_df = pd.read_csv(MEDIA_COMPONENTS)
media_df['Min_Dilution_Factor'] = (media_df['Master Solution Concentration [M]']*1e3)/(media_df['Max Concentration [mM]'])
#display(media_df)

In [7]:
#Create SRC Plate
columns = ['Plate','Well'] + [c for c in media_df['Media Components']] + ['Volume','Target']
df = pd.DataFrame(columns=columns)

#Work in mM concentrations
well = 1
for i,component in media_df.iterrows():    
    if component['Media Components'] in EXTRA_WELLS:
        for _ in range(EXTRA_WELLS[component['Media Components']]):
            row = {'Plate':'src_plate',
                   'Well':well,
                   'Volume':REAGENT_VOLUME,
                   'Target':False,
                   component['Media Components']:component['Master Solution Concentration [M]']*1e3,
                  }
            df = df.append(row, ignore_index=True)
            well +=1 
    else:
        row = {'Plate':'src_plate',
               'Well':well,
               'Volume':REAGENT_VOLUME,
               'Target':False,
               component['Media Components']:component['Master Solution Concentration [M]']*1e3,
              }
        df = df.append(row, ignore_index=True)
        well +=1
df = df.fillna(0)


In [8]:
# Create DataFrame to Help Make Reagent Plate By Hand!
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

well_df = df.loc[:,~df.columns.isin(['Target','Volume'])].melt(id_vars=['Plate','Well'])
well_df = well_df.loc[well_df['value']!=0].rename(columns={'variable':'Reagent'})#.set_index(['Plate','Well'])['Reagent'].to_frame()
well_df = well_df.rename(columns={'Well':'Well Number'})

#
well = lambda x: '{}{}'.format(letters[int((x-1)/12)],(x-1)%12 + 1)
well_df['Well'] = well_df['Well Number'].apply(well)
well_df = well_df[['Plate','Well','Well Number','Reagent']]
display(well_df)

,Plate,Well,Well Number,Reagent
0,src_plate,A1,1,Na2HPO4
1,src_plate,A2,2,Na2HPO4
22,src_plate,A3,3,KH2PO4
23,src_plate,A4,4,KH2PO4
44,src_plate,A5,5,NaCl
45,src_plate,A6,6,NaCl
46,src_plate,A7,7,NaCl
67,src_plate,A8,8,(NH4)2SO4
88,src_plate,A9,9,CaCl2
109,src_plate,A10,10,FeSO4


In [9]:
#Create Water Plate
for i in range(96):
    row = {'Plate':'water_plate',
           'Well':i+1,
           'Volume':WATER_VOLUME,
           'Target':False,
          }
    df = df.append(row, ignore_index=True)
df = df.fillna(0)

In [10]:
#Create Combinatorial Media To Test From Plate
n=len(media_df)
print(n)
media_array = lhs(n,NUM_MEDIA).tolist()
m_df = media_df
media_molarity = [((m_df['Max Concentration [mM]'] - m_df['Min Concentration [mM]'])* m + m_df['Min Concentration [mM]']).tolist() for m in media_array]  
#print(media_molarity)

media_data = media_molarity
#non_zero_index = -2 #All Zeros
#for i,media in enumerate(media_molarity):
#    if i % 4 == 0:
#        non_zero_index += 1
#        
#    buffer = np.zeros(3)
#    if non_zero_index >= 0:
#        buffer[non_zero_index] = media[0]
#        
#    media_data.append(np.concatenate([buffer,media[1:]]).tolist())

13


In [11]:
#Create Robolector Plate
for i,media in enumerate(media_data):
    base_well = (math.floor((i)/8)*24 + (i) % 8) + 1
    for j in range(3):
        row = ['dest_plate',base_well + 8*j] + media_data[i] + [WELL_VOLUME,True]
        df = df.append(dict(zip(columns,row)),ignore_index=True)

In [12]:
#Add Dodecane Overlay (If Needed...)
if OVERLAY_VOLUME > 0:
    pass

In [13]:
#Sort DF Columns By Concentration
sorted_components = media_df.sort_values('Min_Dilution_Factor')['Media Components']
column_reorder = ['Plate','Well'] + sorted_components.tolist() + ['Volume','Target']
df = df[column_reorder]

In [14]:
#Write out Initial Media Configuration
df.to_csv(OUTFILE,index=False)
well_df.to_csv(DECKFILE,index=False)